## Orlando Viera
#### April 30, 2021

In [55]:
import pandas as pd
from sqlalchemy import create_engine
import datetime

In [56]:
# Read csv file into dataframe 
df = pd.read_csv('users.csv')
df.columns = [c.lower() for c in df.columns]
df['id'] = df['id'].str.replace(" ", "")                # remove space in the middle of id string
df['visit_date'] = pd.to_datetime(df['visit_date'])     # convert datatype to datetime

In [57]:
# Gather credentials
server_name = input('Enter database server name: ')
database_name = input('Enter database name: ')
user_name = input('Enter database user name: ')
pswd = input('Enter database password:')

Enter database server name: localhost
Enter database name: phoenix
Enter database user name: postgres
Enter database password:olleh5


In [58]:
# Create connection string
conn_string = 'postgresql://' + user_name + ':' + pswd + '@' + server_name + ':5432/'+ database_name

In [59]:
# Start Postgres connection
engine = create_engine(conn_string)

In [60]:
# Create 'users' table in database with data from dataframe 
df.to_sql("users", engine, if_exists="replace", index=False)

In [61]:
# Add year, month, and day to dataframe from visit_date
df['year'] = df['visit_date'].dt.year
df['month'] = df['visit_date'].dt.month
df['day'] = df['visit_date'].dt.day
df.head()

,id,first_name,last_name,age,gender,visit_date,year,month,day
0,163512032040,Leroy,Dillon,55,M,2020-09-07,2020,9,7
1,169801046773,Macaulay,Griffin,14,M,2020-09-07,2020,9,7
2,162306250982,Phillip,Chapman,45,M,2020-09-07,2020,9,7
3,161802062123,Phillip,Moses,39,M,2020-09-07,2020,9,7
4,169806218401,Tatyana,Goodman,40,F,2020-09-07,2020,9,7


In [62]:
# Create aggregate column based on count of id
count_by_day = {'id':'count'}
daily_user_counts=df.groupby(['year','month','day'], as_index=False).agg(count_by_day)
daily_user_counts.columns=['year','month','day','observed']

In [63]:
# Create a rolling 2 week average for the users expected
daily_user_counts['count'] = daily_user_counts.iloc[:, 3].rolling(window=2,min_periods=1).mean()

In [64]:
# Create the 'daily_user_counts' table in the database
daily_user_counts.to_sql("daily_user_counts", engine, if_exists="replace", index=False)

In [65]:
# Read the 'users' table from database to verify data 
users_df = pd.read_sql_query('''select * from users''', con = engine)

In [66]:
users_df

,id,first_name,last_name,age,gender,visit_date
0,163512032040,Leroy,Dillon,55,M,2020-09-07
1,169801046773,Macaulay,Griffin,14,M,2020-09-07
2,162306250982,Phillip,Chapman,45,M,2020-09-07
3,161802062123,Phillip,Moses,39,M,2020-09-07
4,169806218401,Tatyana,Goodman,40,F,2020-09-07
...,...,...,...,...,...,...
93,166409099774,Kadeem,Lindsay,65,M,2020-09-16
94,168911030453,Jermaine,Love,56,M,2020-09-16
95,165107209487,Gemma,Baird,6,F,2020-09-16
96,164306291396,Brent,Barr,67,M,2020-09-16


In [67]:
# Read the 'daily_user_counts' table from database to verify data
daily_user_counts_df = pd.read_sql_query('''select * from daily_user_counts''', con = engine)

In [68]:
daily_user_counts_df

,year,month,day,observed,count
0,2020,9,7,6,6.0
1,2020,9,8,2,4.0
2,2020,9,9,15,8.5
3,2020,9,10,20,17.5
4,2020,9,11,14,17.0
5,2020,9,14,10,12.0
6,2020,9,15,12,11.0
7,2020,9,16,19,15.5
